In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json

2023-07-10 09:29:21.368153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 09:29:22.098474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATA_FOLDER = "/mnt/e/sign-lang-data-2"

In [3]:
# load tf lite model

In [4]:
interpreter = tf.lite.Interpreter(model_path="model_save/model.tflite")
interpreter.allocate_tensors()
model = interpreter.get_signature_runner()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['inputs'], 'outputs': ['outputs']}}

In [6]:
selected_colomns = pd.read_json("inference_args.json", orient="index")
selected_colomns = selected_colomns.transpose().values
selected_colomns = np.reshape(selected_colomns, (164,))

In [7]:
# load test data
test_csv = pd.read_csv(f"{DATA_FOLDER}/train.csv")

In [8]:
x, y = test_csv.iloc[15]["path"], test_csv.iloc[15]["phrase"]

In [9]:
# load parquet file
test_parquet = pd.read_parquet(f"{DATA_FOLDER}/{x}")

In [11]:
model.get_input_details()

{'inputs': {'name': 'serving_default_inputs:0',
  'index': 0,
  'shape': array([  1, 164], dtype=int32),
  'shape_signature': array([ -1, 164], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}}

In [12]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 350,
  'shape': array([32, 62], dtype=int32),
  'shape_signature': array([32, 62], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [13]:
# Read Character to Ordinal Encoding Mapping
with open(f'{DATA_FOLDER}/character_to_prediction_index.json') as json_file:
    CHAR2ORD = json.load(json_file)

# Ordinal to Character Mapping
ORD2CHAR = {j:i for i,j in CHAR2ORD.items()}

# Output Predictions to string
def outputs2phrase(outputs):
    if outputs.ndim == 2:
        outputs = np.argmax(outputs, axis=1)

    return ''.join([ORD2CHAR.get(s, '') for s in outputs])

In [14]:
demo_sequence_id = test_parquet.index.unique()[0]
print(f'demo_sequence_id: {demo_sequence_id}')
demo_raw_data = test_parquet.loc[demo_sequence_id, selected_colomns].values
demo_phrase_true = y
print(f'demo_raw_data shape: {demo_raw_data.shape}, dtype: {demo_raw_data.dtype}')
demo_output = model(inputs=demo_raw_data)['outputs']
print(f'demo_output shape: {demo_output.shape}, dtype: {demo_output.dtype}')
print(f'demo_outputs phrase decoded: {outputs2phrase(demo_output)}')
print(f'phrase true: {demo_phrase_true}')

demo_sequence_id: 1816796431
demo_raw_data shape: (123, 164), dtype: float32
demo_output shape: (32, 62), dtype: float32
demo_outputs phrase decoded: 3 creek house
phrase true: wildberries_ru


In [15]:
#demo_outputs phrase decoded: 303 corthes sth streeeee
#phrase true: 3 creekhouse